In [1]:
import os
from transformers import PreTrainedTokenizerFast
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace, Sequence, Split, PreTokenizer
from tokenizers.trainers import BpeTrainer
from tokenizers.processors import TemplateProcessing
import datasets

In [2]:
ds = datasets.load_dataset("jablonkagroup/pubchem-smiles-molecular-formula", split="train")

Resolving data files:   0%|          | 0/21 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/21 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/118536656 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/21 [00:00<?, ?it/s]

In [3]:
ds

Dataset({
    features: ['CID', 'smiles', 'molecular_formula'],
    num_rows: 118536656
})

In [4]:
smiles_tokenizer = PreTrainedTokenizerFast.from_pretrained("DeepChem/SmilesTokenizer_PubChem_1M")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/420 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/672 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RobertaTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [5]:
def create_smiles_pre_tokenizer():
    """Create a pre-tokenizer suitable for SMILES strings."""
    # SMILES use specific characters - we'll split on common atom/bond boundaries
    # This helps the tokenizer learn meaningful chemical substructures
    return Sequence([
        Split(pattern=r'(\[[^\]]+]|Br?|Cl?|N|O|S|P|F|I|b|c|n|o|s|p|\(|\)|\.|=|#|-|\+|\\|\/|:|~|@|\?|>>?|\*|\$|\%[0-9]{2}|[0-9])', behavior="isolated"),
    ])

In [6]:
# Define special tokens
special_tokens = [
    "[PAD]",    # Padding token
    "[UNK]",    # Unknown token  
    "[CLS]",    # Classification token (start of sequence)
    "[SEP]",    # Separator token (end of sequence)
]

# Initialize tokenizer with BPE model
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

tokenizer.pre_tokenizer = create_smiles_pre_tokenizer()

trainer = BpeTrainer(
    # vocab_size=1024,
    special_tokens=special_tokens,
    min_frequency=10000,
    show_progress=True
)

In [ ]:
# Train the tokenizer
tokenizer.train_from_iterator(ds["smiles"], trainer=trainer)


In [20]:
# Add post-processor to add special tokens
tokenizer.post_processor = TemplateProcessing(
    single="[CLS] $A [SEP]",
    special_tokens=[
        ("[CLS]", tokenizer.token_to_id("[CLS]")),
        ("[SEP]", tokenizer.token_to_id("[SEP]")),
    ],
)
    
# Create transformers tokenizer
fast_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
)

output_dir = "/mnt/workspace/md4/data"
tokenizer_name = "pubchem_large_tokenizer"
# Save tokenizer
os.makedirs(output_dir, exist_ok=True)
tokenizer_path = os.path.join(output_dir, tokenizer_name)
fast_tokenizer.save_pretrained(tokenizer_path)

print(f"Tokenizer saved to {tokenizer_path}")
print(f"Vocabulary size: {fast_tokenizer.vocab_size}")

Tokenizer saved to /mnt/workspace/md4/data/pubchem_large_tokenizer
Vocabulary size: 1024


In [29]:
fast_tokenizer.convert_ids_to_tokens(fast_tokenizer.encode(ds["smiles"][0]))

['[CLS]', 'CCCC', 'CCN', 'O', 'P', '(C)', '(=O)', 'OC', '[SEP]']